In [1]:
import sys
sys.path.append('.\\')

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# RNN
from keras.models import Sequential  
from keras.layers import Dense, GRU
from keras.layers import Input

import helper_functions

# Extract data

In [3]:
train_data = helper_functions.load_data()

In [29]:
# Data is normalised, so each input has the same shape
input_shape = train_data["mfccs"][0].shape
print(f'mel-log data shape: {input_shape}')

mel-log data shape: (15, 173)


In [5]:
n_classes = len(train_data['classes'][0])
n_classes

12

In [84]:
x = np.array(train_data['mfccs'])
y = np.array([helper_functions.one_hot_to_label(label) for label in train_data['classes']])
y_train_one_hot = np.array(train_data['classes'])
n_dim = x_train.shape[2]

x_train shape: (64721, 15, 173)
y_train shape: (64721,)
n_dim: 173


In [98]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y_train_one_hot, test_size=0.2)

In [99]:
print(f'x_train shape: {x_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'n_dim: {n_dim}')

x_train shape: (51776, 15, 173)
y_train shape: (51776, 12)
n_dim: 173


# Build the model

In [68]:
from keras.layers import GlobalMaxPooling1D, Dropout, BatchNormalization
from keras.optimizers import Adam

In [100]:
model = Sequential()
model.add(Input(shape=input_shape))
model.add(BatchNormalization(axis=2))
model.add(GRU(n_dim*2, return_sequences=True, dropout=0.01, recurrent_dropout=0.1, activation='relu'))  
# model.add(GRU(50*2, dropout=0.05, recurrent_dropout=0.15, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(GlobalMaxPooling1D(input_shape=(None, 15, 173)))
# model.add(GRU(n_dim, return_sequences=True, dropout=0.1, recurrent_dropout=0.2, activation='relu'))  
# model.add(Dropout(0.2))
# model.add(GRU(n_dim, dropout=0.1, recurrent_dropout=0.2))  
# model.add(Dense(15, activation='relu'))  
model.add(Dense(n_classes, activation='softmax'))

In [101]:
# model compilation  
optimizer = Adam(learning_rate=1e-3)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [102]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_5 (Batch (None, 15, 173)           692       
_________________________________________________________________
gru_55 (GRU)                 (None, 15, 346)           540798    
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 346)           0         
_________________________________________________________________
global_max_pooling1d_16 (Glo (None, 346)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 12)                4164      
Total params: 545,654
Trainable params: 545,308
Non-trainable params: 346
_________________________________________________________________


In [103]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_val, y_val))

Epoch 1/5
809/809 [==============================] - 138s 156ms/step - loss: 1.3643 - accuracy: 0.6450 - val_loss: 0.7788 - val_accuracy: 0.7540
Epoch 2/5
809/809 [==============================] - 121s 149ms/step - loss: 0.7312 - accuracy: 0.7634 - val_loss: 0.6107 - val_accuracy: 0.8061
Epoch 3/5
809/809 [==============================] - 116s 144ms/step - loss: 0.5551 - accuracy: 0.8168 - val_loss: 0.4859 - val_accuracy: 0.8377
Epoch 4/5
809/809 [==============================] - 126s 156ms/step - loss: 0.4633 - accuracy: 0.8481 - val_loss: 0.4572 - val_accuracy: 0.8512
Epoch 5/5
809/809 [==============================] - 122s 151ms/step - loss: 0.4088 - accuracy: 0.8653 - val_loss: 0.4126 - val_accuracy: 0.8674


# Save model

In [110]:
model.save('.\\models\\gru')

INFO:tensorflow:Assets written to: .\models\gru\assets
